In [ ]:
# Compare magic_duckdb against baseline (DuckDB) and other implementations
#
# Create a semi-large dataframe

from pandas import DataFrame
import numpy as np

num_rows = 1000000

simpledf = DataFrame(np.random.randn(num_rows,20))

In [ ]:
# DuckDB

display("Native duckdb")
import duckdb
with duckdb.connect() as con:
    %timeit con.sql("select * from simpledf").df()

In [ ]:
%load_ext magic_duckdb

display("magic_duckdb: pandas")

# First test is Pandas. The -t df is not needed, since Pandas is the default.
%dql -t df
%timeit odf = %dql select * from simpledf

display("magic_duckdb: arrow")
%dql -t arrow
%timeit %dql select * from simpledf


In [ ]:
display("jupysql, duckdb_engine, sql_alchemy")

%load_ext sql
%sql duckdb:///:memory:
%config SqlMagic.autopandas = True
%timeit %sql select * from simpledf
